# Timothy Berrilll  
## CMPE 258 Assignment 3  
## Question 1  

### Section 0 - Setting up Colab

In [1]:
import tensorflow as tf
import numpy as np

### Section 1 - Building the YOLO Model

In [38]:
# create 100 fake images to use 
fake_data = np.random.randn(100, 127, 127, 3)


# OUTPUT SIZE
grid = 8
BBOXES = 2
NUM_CLASSES = 20
OUTPUT_SIZE = grid * grid * (BBOXES * (5 + NUM_CLASSES))

The output size for the model should be (grid, grid, number of bounding boxes * (5 + number of classes))

The grid is 8 x 8, and each grid square has 2 anchor boxes. For each anchor box, there are 5 outputs to describe the location, and one output for each of the target classes. So in this case, there are 25 parameters per bounding box and 2 bounding boxes per grid square -> 50 outputs per grid square. The output of the model should then look like (8, 8, 50)

In [39]:
def get_yolo_like_model(input_size):

  inputs = tf.keras.Input(shape=input_size)
  x = inputs

  # block 1
  x = tf.keras.layers.Conv2D(64, kernel_size=7, padding='same', strides=2)(x)
  x = tf.keras.layers.MaxPool2D(2, padding='same', strides=2)(x)

  # block 2
  x = tf.keras.layers.Conv2D(192, padding='same', kernel_size=3)(x)
  x = tf.keras.layers.MaxPool2D(2, padding='same', strides=2)(x)

  # block 3
  x = tf.keras.layers.Conv2D(128, padding='same', kernel_size=1)(x)
  x = tf.keras.layers.Conv2D(256, padding='same', kernel_size=3)(x)
  x = tf.keras.layers.MaxPool2D(2, padding='same', strides=2)(x)

  # block 4
  x = tf.keras.layers.Conv2D(512, padding='same', kernel_size=1)(x)
  x = tf.keras.layers.Conv2D(512, padding='same', kernel_size=3)(x)

  # last block
  x = tf.keras.layers.Conv2D(512, kernel_size=3)(x)
  x = tf.keras.layers.Conv2D(512, kernel_size=3)(x)

  # output
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(512, activation='relu')(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dense(OUTPUT_SIZE, activation='sigmoid')(x)
  output = tf.keras.layers.Reshape(target_shape=(grid, grid, BBOXES * (5 + NUM_CLASSES)))(x)
  return tf.keras.models.Model(inputs=[inputs], outputs=[output])

In [40]:
yolo_model = get_yolo_like_model(input_size=(127, 127, 3))
yolo_model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 127, 127, 3)]     0         
                                                                 
 conv2d_126 (Conv2D)         (None, 64, 64, 64)        9472      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_127 (Conv2D)         (None, 32, 32, 192)       110784    
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 16, 16, 192)      0         
 g2D)                                                            
                                                                 
 conv2d_128 (Conv2D)         (None, 16, 16, 128)       2470